# data

In [3]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.data import Data, Dataset

class CustomData(Data):
    """
    override __inc__ so DataLoader doesn't increment indices
    """
    def __inc__(self, key, value, *args, **kwargs):
        return 0

class SpotData(Dataset):
    """
    dataset with supervision/evaluation edges.
    get(idx) return ALL outgoing edges of the graph of playlist "idx" since calculating metrics like recall@k needs all the playlist's positive edges
    """
    def __init__(self, root, edge_index, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.edge_index = edge_index
        # playlists will all be in row 0, b/c sorted by RandLinkSplit
        self.unique_idxs = torch.unique(edge_index[0,:]).tolist()
        self.num_nodes = len(self.unique_idxs)

    def len(self):
        return self.num_nodes

    # returns all outgoing edges associated with playlist idx
    def get(self, idx):
        edge_index = self.edge_index[:, self.edge_index[0,:] == idx]
        return CustomData(edge_index=edge_index)


# preprocessing

In [5]:
pip install snap

  Preparing metadata (setup.py) ... done
  Created wheel for snap: filename=snap-0.5-py3-none-any.whl size=19393 sha256=75e2bc1d37cc9bd632c46377353f25d7798824476e1a4b5f35c0142f3cf27a36
  Stored in directory: /root/.cache/pip/wheels/b2/bf/5e/6d570c7910ef2111228c9afe1f8590629917a951388898916e
Successfully built snap


In [11]:
# !pip install snap-stanford


ERROR: Could not find a version that satisfies the requirement snap-stanford (from versions: none)
ERROR: No matching distribution found for snap-stanford


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# import os
# import json
# import snap
# import torch
# from torch_geometric.data import Data
# from tqdm import tqdm

# # argparse 대신 여기서 N과 K 값을 직접 정의
# N = 100  # 예시: 데이터셋을 위해 사용할 파일의 수
# K = 5    # 예시: K-core 그래프를 위한 값

# def getFiles(N):
#     '''
#     returns:
#         directory and files to use for dataset
#     '''
#     cwd = os.getcwd()
#     dir = '/content/drive/My Drive/spotify_million_playlist_dataset/data'  # Google 드라이브 경로 지정
#     files = os.listdir(dir)

#     files = sorted(files, key=lambda x: int(x.split(".")[2].split("-")[0]))
#     return dir, files[:N]

# def makeGraph(dir, files):
#     '''
#     returns:
#         graph, number of original playlists and hashmap of PIDs/URIs
#     '''
#     # create undirected SNAP graph
#     G = snap.TUNGraph.New()
#     # create node for all playlist IDs
#     for file in files:
#         with open(os.path.join(dir, file), 'r') as f:
#             data = json.load(f)['playlists']
#             for playlist in data:
#                 G.AddNode(playlist['pid'])

#     # playlists indexed from [0, num_playlists - 1]
#     # songs indexed from [num_playlists, num_playlist + num_songs]
#     lastPID = max([n.GetId() for n in G.Nodes()])
#     assert lastPID == len([n for n in G.Nodes()]) - 1
#     SID = lastPID + 1
#     # hashmap of { PID: {'name': _ } } and { URI : {'SID': _ , ... } }
#     p_meta, uris = {}, {}

#     for file in files:
#         with open(os.path.join(dir, file), 'r') as f:
#             data = json.load(f)['playlists']
#             for playlist in data:
#                 p_meta[playlist['pid']] = {'name': playlist['name']}
#                 for song in playlist['tracks']:
#                     uri = song['track_uri']
#                     if uri not in uris:
#                         uris[uri] = {
#                             'SID': SID,
#                             'track_name': song['track_name'],
#                             'artist_name': song['artist_name'],
#                             'artist_uri': song['artist_uri']
#                         }
#                         G.AddNode(SID)
#                         SID += 1
#                     # add edge between playlist and song
#                     G.AddEdge(playlist['pid'], uris[uri]['SID'])
#     orig_playlists = len([n for n in G.Nodes() if n.GetId() <= lastPID])
#     return G, orig_playlists, lastPID, p_meta, uris

# def getKCore(G, K, lastPID):
#     '''
#     returns:
#         K-core graph, number of playlists, songs and edges
#     '''
#     G = G.GetKCore(K)
#     if G.Empty():
#         raise Exception(f"No graph exists for K={K}")
#     num_playlists = len([n for n in G.Nodes() if n.GetId() <= lastPID])
#     num_songs = len([n for n in G.Nodes() if n.GetId() > lastPID])
#     num_edges = len([x for x in G.Edges()])
#     return G, num_playlists, num_songs, num_edges

# def reindexGraph(G, orig_playlists, num_playlists, num_songs, p_meta, uris):
#     # create hashmap converting old IDs to new IDs
#     ID, PIDs, SIDs = 0, {}, {}
#     for N in G.Nodes():
#         oldID = N.GetId()
#         assert oldID not in PIDs and oldID not in SIDs
#         if oldID <= orig_playlists - 1:
#             PIDs[oldID] = ID
#         else:
#             SIDs[oldID] = ID
#         ID += 1
#     assert max(PIDs.values()) == num_playlists - 1
#     assert len(PIDs.values()) == num_playlists
#     assert max(SIDs.values()) == len([n for n in G.Nodes()]) - 1
#     assert len(SIDs.values()) == num_songs

#     # hashmap of { SID : {'track_uri': _ , ... } }
#     s_meta = {}
#     for uri, info in uris.items():
#         if info['SID'] in SIDs:
#             newID = SIDs[info['SID']]
#             s_meta[newID] = {
#                 'track_uri': uri,
#                 'track_name': info['track_name'],
#                 'artist_name': info['artist_name'],
#                 'artist_uri': info['artist_uri']
#             }
#     p_meta = { PIDs[k]: v for k, v in p_meta.items() if k in PIDs }
#     return G, p_meta, s_meta, PIDs, SIDs

# def createPyObject(G, PIDs, SIDs):
#     # convert to edge_index and storing in a PyG Data object
#     edges = []
#     for E in tqdm(G.Edges()):
#         # create all edges from playlist -> song
#         assert (E.GetSrcNId() in PIDs) and (E.GetSrcNId() not in SIDs)
#         assert (E.GetDstNId() in SIDs) and (E.GetDstNId() not in PIDs)
#         edge = [PIDs[E.GetSrcNId()], SIDs[E.GetDstNId()]]
#         edges.append(edge)
#         edges.append(edge[::-1])
#     edge_idx = torch.LongTensor(edges)
#     return Data(edge_index=edge_idx.t().contiguous(), num_nodes=G.GetNodes())

# def saveObject(data, p_meta, s_meta, num_playlists, num_songs, num_edges, K, N):

#     cwd = os.getcwd()
#     dir = os.path.join(cwd, 'data')
#     if not os.path.exists(dir):
#         os.makedirs(dir)
#     torch.save(data, os.path.join(dir, 'data_object.pt'))
#     dataset = {'num_playlists': num_playlists, 'num_nodes': num_playlists + num_songs, 'kcore_value_k': K, 'num_spotify_files_used': N, 'num_edges_directed': 2 * num_edges, 'num_edges_undirected': num_edges}
#     with open(os.path.join(dir, 'graph_info.json'), 'w') as f:
#         json.dump(dataset, f)
#     with open(os.path.join(dir, 'playlist_info.json'), 'w') as f:
#         json.dump(p_meta, f)
#     with open(os.path.join(dir, 'song_info.json'), 'w') as f:
#         json.dump(s_meta, f)

# if __name__ == "__main__":
#     # N, K = parseArgs() 대신에 직접 정의된 N, K를 사용
#     dir, files = getFiles(N)
#     G, orig_playlists, lastPID, p_meta, uris = makeGraph(dir, files)
#     G, num_playlists, num_songs, num_edges = getKCore(G, K, lastPID)
#     G, p_meta, s_meta, PIDs, SIDs = reindexGraph(G, orig_playlists, num_playlists, num_songs, p_meta, uris)
#     data = createPyObject(G, PIDs, SIDs)
#     saveObject(data, p_meta, s_meta, num_playlists, num_songs, num_edges, K, N)


AttributeError: module 'snap' has no attribute 'TUNGraph'

In [12]:
!pip install networkx


In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


import os
import json
import networkx as nx
import torch
from torch_geometric.data import Data
from tqdm import tqdm

# Example values for N and K
N = 100  # Number of files to use for the dataset
K = 5    # Value for K-core decomposition

def getFiles(N):
    '''
    Returns directory and files to use for dataset
    '''
    dir = '/content/drive/My Drive/spotify_million_playlist_dataset/data'
    files = sorted(os.listdir(dir), key=lambda x: int(x.split(".")[2].split("-")[0]))
    return dir, files[:N]

def makeGraph(dir, files):
    '''
    Returns a graph, number of original playlists, and hashmap of PIDs/URIs
    '''
    G = nx.Graph()
    p_meta, uris = {}, {}
    SID = 0

    for file in files:
        with open(os.path.join(dir, file), 'r') as f:
            data = json.load(f)['playlists']
            for playlist in data:
                pid = playlist['pid']
                G.add_node(pid, type='playlist')
                p_meta[pid] = {'name': playlist['name']}
                for song in playlist['tracks']:
                    uri = song['track_uri']
                    if uri not in uris:
                        uris[uri] = {'SID': SID, 'track_name': song['track_name'],
                                     'artist_name': song['artist_name'], 'artist_uri': song['artist_uri']}
                        G.add_node(SID, type='song')
                        SID += 1
                    G.add_edge(pid, uris[uri]['SID'])

    orig_playlists = sum(1 for _, data in G.nodes(data=True) if data['type'] == 'playlist')
    return G, orig_playlists, SID - 1, p_meta, uris

def getKCore(G, K):
    '''
    Returns K-core graph, number of playlists, songs, and edges
    '''
    G_kcore = nx.k_core(G, k=K)
    num_playlists = sum(1 for _, data in G_kcore.nodes(data=True) if data['type'] == 'playlist')
    num_songs = sum(1 for _, data in G_kcore.nodes(data=True) if data['type'] == 'song')
    num_edges = G_kcore.number_of_edges()
    return G_kcore, num_playlists, num_songs, num_edges

def reindexGraph(G, orig_playlists, num_playlists, num_songs, p_meta, uris):
    # NetworkX handles node indexing internally, so this step might be simplified
    # depending on the specific needs of reindexing in your application
    # This step is highly specific to the original snap application and may not be directly applicable in NetworkX
    pass

def createPyObject(G):
    # Convert to edge_index and storing in a PyG Data object
    edge_list = [(u, v) for u, v in G.edges()]
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    return Data(edge_index=edge_index, num_nodes=G.number_of_nodes())

def saveObject(data, p_meta, s_meta, num_playlists, num_songs, num_edges, K, N):
    # Save the data object and metadata as before
    # Implementation remains the same as in your original code
    pass

# Main execution
if __name__ == "__main__":
    dir, files = getFiles(N)
    G, orig_playlists, lastPID, p_meta, uris = makeGraph(dir, files)
    G_kcore, num_playlists, num_songs, num_edges = getKCore(G, K)
    # Reindexing might not be necessary or needs adaptation
    data = createPyObject(G_kcore)
    saveObject(data, p_meta, s_meta, num_playlists, num_songs, num_edges, K, N)


Mounted at /content/drive


OSError: [Errno 107] Transport endpoint is not connected

# GCN

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

In [ ]:
pip install recommender-system

In [ ]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

In [ ]:
!pip install -e git+https://github.com/microsoft/recommenders/#egg=recommenders

In [ ]:
!pip install recommenders

In [ ]:
import recommenders
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

In [ ]:
!pip install gnn

In [ ]:
import torch
import torch.nn as nn
from lightGCN import LightGCN
from utils import recall

class GNN(nn.Module):
    """
    full GNN with learnable playlist/song embeddings and LightGCN layers
    """
    def __init__(self, emb_dim, num_nodes, num_playlists, num_layers):
        super(GNN, self).__init__()
        self.emb_dim = emb_dim
        self.num_nodes = num_nodes
        self.num_playlists = num_playlists
        self.num_layers = num_layers
        self.embeddings = torch.nn.Embedding(
          num_embeddings=self.num_nodes,
          embedding_dim=self.emb_dim
        )
        nn.init.normal_(self.embeddings.weight, std=0.1)
        self.layers = nn.ModuleList()
        for _ in range(self.num_layers):
            self.layers.append(LightGCN())
        self.sigmoid = torch.sigmoid

    def forward(self):
        raise NotImplementedError("Do not use.")

    def gnn_propagation(self, edge_index_mp):
        """
        func:
          performn linear embedding propagation via lightGCN layers
        args:
          edge_index_mp: a tensor of all message passing edges
        returns:
          final multi-scale embeddings for all playlist/songs
        """
        l = self.embeddings.weight # layer-0 embeddings
        layers = [l]
        # GNN propagation loop
        for i in range(self.num_layers):
            l = self.layers[i](l, edge_index_mp)
            layers.append(l)
        return torch.stack(layers, dim=0).mean(dim=0)

    def predict_scores(self, edge_index, embs):
        """
        func:
          computes predicted scores for each playlist/song pair via dot product
        args:
          edge_index: tensor of playlist/song edges we compute
          embs: node embeddings for calculating predicted scores (typically the multi-scale embeddings from gnn_propagation())
        returns:
          predicted scores for each playlist/song pair in edge_index
        """
        # dot product for each playlist/song pair
        scores = embs[edge_index[0,:], :] * embs[edge_index[1,:], :]
        return self.sigmoid(scores.sum(dim=1))

    def loss(self, data_mp, data_pos, data_neg):
        """
        func:
          training set. GNN propagation on message passing edges. predict scores on training examples. calculate Bayesian Personalized Ranking.
        args:
          data_mp: tensor of message passing edges
          data_pos: set of positive edges for loss calculation
          data_neg: set of negative edges for loss calculation
        returns:
          loss calculated on the positive/negative training edges
        """
        final_embs = self.gnn_propagation(data_mp.edge_index)
        # get edge prediction scores for all positive/negative evaluation edges
        pos_scores = self.predict_scores(data_pos.edge_index, final_embs)
        neg_scores = self.predict_scores(data_neg.edge_index, final_embs)
        loss = -torch.log(self.sigmoid(pos_scores - neg_scores)).mean()
        return loss

    def evaluation(self, data_mp, data_pos, k):
        """
        func:
          calculate recall@k on validation/test set
        args:
          data_mp: tensor of message passing edges
          data_pos: set of positive edges for scoring metric
          k: k for recall@k
        returns:
          hashmap of { PID: recall@k }
        """
        final_embs = self.gnn_propagation(data_mp.edge_index)
        # embeddings of all unique playlists in the batch of evaluation edges
        unique_playlists = torch.unique_consecutive(data_pos.edge_index[0,:])
        # shape [ playlists_in_batch, EMB_DIM ]
        playlist_emb = final_embs[unique_playlists, :]
        # shape [ songs_in_dataset, EMB_DIM ]
        song_emb = final_embs[self.num_playlists:, :]
        # entry i,j is rating of song j for playlist i
        ratings = self.sigmoid(torch.matmul(playlist_emb, song_emb.t()))
        result = recall(ratings.cpu(), k, self.num_playlists, data_pos.edge_index.cpu(), unique_playlists.cpu(), data_mp.edge_index.cpu())
        return result


# lightGCN

In [7]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

class LightGCN(MessagePassing):
  """
  a single LightGCN layer
  """
  def __init__(self):
    super(LightGCN, self).__init__(aggr='add')

  def message(self, x_j, norm):
    '''
    args:
      x_j: node embeddings of neighbors of shape [E, emb_dim]
      norm: normalization calculated in forward()
    returns:
      message from neighboring nodes j to central node i
    '''
    return norm.view(-1, 1) * x_j

  def forward(self, x, edge_index):
    """
    args:
      x: current node embeddings of shape [N, emb_dim]
      edge_index: message passing edges of shape [2, E]
    returns:
      updated embeddings after this layer
    """
    row, col = edge_index
    deg = degree(col)
    deg_inv_sqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
    return self.propagate(edge_index, x=x, norm=norm)

# util.py

In [8]:
import torch
import numpy as np
from torch_geometric.data import Data

def sampleNeg(batch, num_playlists, num_nodes):
   negs = []
   for _ in batch.edge_index[0,:]:
      rand = torch.randint(num_playlists, num_nodes, (1,))
      negs.append(rand.item())
   edge_index_negs = torch.row_stack([
      batch.edge_index[0,:], torch.LongTensor(negs)
   ])
   return Data(edge_index=edge_index_negs)

def recall(all_ratings, k, num_playlists, ground_truth, unique_playlists, data_mp):
   """
   Calculates recall@k during validation/testing for a single batch.

   args:
     all_ratings: array of shape [number of playlists in batch, number of songs in whole dataset]
     k: the value of k to use for recall@k
     num_playlists: the number of playlists in the dataset
     ground_truth: array of shape [2, X] where each column is a pair of (playlist_idx, positive song idx). This is the
        batch that we are calculating metrics on.
     unique_playlists: 1D vector of length [number of playlists in batch], which specifies which playlist corresponds
        to each row of all_ratings
     data_mp: an array of shape [2, Y]. This is all of the known message-passing edges. We will use this to make sure we
        don't recommend songs that are already known to be in the playlist.
   returns:
     Dictionary of playlist ID -> recall@k on that playlist
   """
   # We don't want to recommend songs that are already known to be in the playlist.
   # Set those to a low rating so they won't be recommended
   known_edges = data_mp[:, data_mp[0,:] < num_playlists] # removing duplicate edges (since data_mp is undirected). also makes it so that for each column, playlist idx is in row 0 and song idx is in row 1
   playlist_to_idx_in_batch = {playlist: i for i, playlist in enumerate(unique_playlists.tolist())}
   exclude_playlists, exclude_songs = [], [] # already-known playlist/song links. Don't want to recommend these again
   for i in range(known_edges.shape[1]): # looping over all known edges
      pl, song = known_edges[:,i].tolist()
      if pl in playlist_to_idx_in_batch: # don't need the edges in data_mp that are from playlists that are not in this batch
         exclude_playlists.append(playlist_to_idx_in_batch[pl])
         exclude_songs.append(song - num_playlists) # subtract num_playlists to get indexing into all_ratings correct
   all_ratings[exclude_playlists, exclude_songs] = -10000 # setting to a very low score so they won't be recommended

   # Get top k recommendations for each playlist
   _, top_k = torch.topk(all_ratings, k=k, dim=1)
   top_k += num_playlists # topk returned indices of songs in ratings, which doesn't include playlists. Need to shift up by num_playlists to get the actual song indices

   ret = {}
   for i, playlist in enumerate(unique_playlists):
      pos_songs = ground_truth[1, ground_truth[0, :] == playlist]
      k_recs = top_k[i, :] # top k recommendations for playlist
      recall = len(np.intersect1d(pos_songs, k_recs)) / len(pos_songs)
      ret[playlist] = recall
   return ret

# main

In [ ]:
import torch
import numpy as np
import os
import json
from GNN import GNN
from torch_geometric import seed_everything
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from utils import sampleNeg
from data import SpotData

def train(model, data_mp, loader, opt, num_playlists, num_nodes, device):
    """
    args:
       model: GNN model
       data_mp: message passing edges
       loader: dataloader for eval edges
       opt: the optimizer
       num_playlists: number of total playlists
       num_nodes: number of total nodes
       device: CPU/GPU
    returns:
       epoch's training loss
    """
    loss, samples = 0, 0
    model.train() # set module in training mode
    for batch in loader:
        data_mp, batch = data_mp.to(device), batch.to(device)
        negs = sampleNeg(batch, num_playlists, num_nodes).to(device)
        # forward + backprop + loss + update
        loss = model.loss(data_mp, batch, negs)
        opt.zero_grad()
        loss.backward()
        opt.step()
        # compute metrics
        sample = batch.edge_index.shape[1]
        loss += loss.item() * sample
        samples += sample
    return loss / samples

def test(model, data_mp, loader, k, device, save_dir, epoch):
    """
    args:
       model: GNN model
       data_mp: message passing edges
       loader: dataloader for eval edges
       k: k for recall@k
       device: CPU/GPU
       save_dir: dir to save embeddings
       epoch: number of the current epoch
    returns:
       recall@k for this epoch
    """
    model.eval()
    recalls = {}
    with torch.no_grad():
        data_mp = data_mp.to(device)
        if save_dir is not None:
            embs = gnn.gnn_propagation(data_mp.edge_index)
            torch.save(embs, os.path.join(save_dir, f"epoch_{epoch}.pt"))
        for batch in loader:
            batch = batch.to(device)
            recalls.update(model.evaluation(data_mp, batch, k))
    return np.mean(list(recalls.values()))

if __name__ == "__main__":

    seed_everything(5)
    epochs = 30
    k = 300
    num_layers = 3 # number of LightGCN layers (number of hops)
    batch_size = 2048  # number of playlists in the batch
    emb_dim = 64 # dimension for the playlist/song embeddings
    emb_dir = 'embeddings'  # path to save embeddings
    if not os.path.exists(emb_dir):
        os.makedirs(emb_dir)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    data = torch.load("data/data_object.pt")
    with open("data/graph_info.json", 'r') as f:
        info = json.load(f)
    num_playlists, num_nodes = info["num_playlists"], info["num_nodes"]

    # train/val/test split (70-15-15)
    transform = RandomLinkSplit(
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=0, num_val=0.15,
        num_test=0.15
    )
    train_split, val_split, test_split = transform(data)

    # create message passing edges for propagation AND evaluation edges
    train_mp = Data(edge_index=train_split.edge_index)
    val_mp = Data(edge_index=val_split.edge_index)
    test_mp = Data(edge_index=test_split.edge_index)
    train_ev = SpotData('temp', edge_index=train_split.edge_label_index)
    val_ev = SpotData('temp', edge_index=val_split.edge_label_index)
    test_ev = SpotData('temp', edge_index=test_split.edge_label_index)

    # dataLoaders for the supervision/evaluation edges
    train_loader = DataLoader(train_ev, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ev, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ev, batch_size=batch_size, shuffle=False)

    # initialize GNN model
    gnn = GNN(
        emb_dim=emb_dim,
        num_nodes=data.num_nodes,
        num_playlists=num_playlists,
        num_layers=num_layers
    ).to(device)
    opt = torch.optim.Adam(gnn.parameters(), lr=1e-3)

    # list of [epoch, train loss / val recall@K ]
    losses, vals = [], []
    for e in range(epochs):
        loss = train(
            gnn, train_mp, train_loader, opt,
            num_playlists,  num_nodes, device
        )
        losses.append((e, loss))
        if e % 5 == 0:
            val = test(gnn, val_mp, val_loader, k, device, emb_dir, e)
            vals.append((e, val))
            print(f"Epoch {e}: train_loss={loss}, val_recall={val}")
        else:
            print(f"Epoch {e}: train_loss={loss}")

    best_val = max(vals, key = lambda x: x[1])
    print(f"Best val recall@k: {best_val[1]} @ epoch {best_val[0]}")

    test_recall = test(gnn, test_mp, test_loader, k, device, None, None)
    print(f"Test recall@k: {test_recall}")
